PCSEA Low Level Interface
==
This notebook demonstrates use of the low level interface to PCSEA.  This interface requires explicit calls to construct the unit cells and perform measurements.  Each set of potential parameters must be explicitly specified as well.

In [1]:
# Import necessary modules
import inspect, io, math, numpy, os, sys
sys.path.append(os.path.realpath(os.path.join(os.path.dirname(inspect.getfile(inspect.currentframe())), "..")))
import crystal, potential

Working with Cells
--
Cells can easily be constructed, written, and read.

In [2]:
# Create new cells directly from vectors and coordinates
NaCl = crystal.Cell(
    numpy.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]), [
    numpy.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0]]),
    numpy.array([[0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1]])])
CsCl = crystal.Cell(
    numpy.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]), [
    numpy.array([[0, 0, 0]]),
    numpy.array([[1, 1, 1]])])

In [3]:
# Retrieve vector data
[NaCl.vector(i) for i in range(NaCl.dimensions())]

[array([ 2.,  0.,  0.]), array([ 0.,  2.,  0.]), array([ 0.,  0.,  2.])]

In [4]:
# Retrieve atom data
[NaCl.atom(i, j) for i in range(NaCl.atom_types()) for j in range(NaCl.atom_count(i))]

[array([ 0.,  0.,  0.]),
 array([ 0.,  1.,  1.]),
 array([ 1.,  0.,  1.]),
 array([ 1.,  1.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  1.,  1.])]

In [5]:
# Convert the cell to a textual representation
string_io = io.StringIO()
NaCl.write_data(string_io)
print(string_io.getvalue())

3 4 4
2.0 0.0 0.0
0.0 2.0 0.0
0.0 0.0 2.0
0.0 0.0 0.0
0.0 1.0 1.0
1.0 0.0 1.0
1.0 1.0 0.0
0.0 0.0 1.0
0.0 1.0 0.0
1.0 0.0 0.0
1.0 1.0 1.0



In [6]:
# Convert the textual representation back to a cell: it is identical
string_io.seek(0)
NaCl_io = crystal.Cell.read_data(string_io)
[NaCl_io.vector(i) for i in range(NaCl_io.dimensions())]

[array([ 2.,  0.,  0.]), array([ 0.,  2.,  0.]), array([ 0.,  0.,  2.])]

In [7]:
[NaCl_io.atom(i, j) for i in range(NaCl_io.atom_types()) for j in range(NaCl_io.atom_count(i))]

[array([ 0.,  0.,  0.]),
 array([ 0.,  1.,  1.]),
 array([ 1.,  0.,  1.]),
 array([ 1.,  1.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  1.,  1.])]

Making Measurements
--
In order to determine the energy of a structure, it is first necessary to make measurements on the atoms within.  Many energy calculations can be performed using a single set of measurements provided that the relative interatomic distances need not change.

In [8]:
# Perform the measurements (fields are updated internally)
NaCl.measure(10)
CsCl.measure(10)

In [9]:
# Histograms have been generated indicating the number of atoms present at a given distance
([NaCl.histogram(i, j) for i in range(NaCl.atom_types()) for j in range(NaCl.atom_types())],
 [CsCl.histogram(i, j) for i in range(CsCl.atom_types()) for j in range(CsCl.atom_types())])

([{1.4142135623730951: 48,
   2.0: 24,
   2.4494897427831779: 96,
   2.8284271247461903: 48,
   3.1622776601683795: 96,
   3.4641016151377544: 32,
   3.7416573867739413: 192,
   4.0: 24,
   4.2426406871192848: 144,
   4.4721359549995796: 96,
   4.6904157598234297: 96,
   4.8989794855663558: 96,
   5.0990195135927845: 288,
   5.4772255750516612: 192,
   5.6568542494923806: 48,
   5.8309518948453007: 192,
   6.0: 120,
   6.164414002968976: 288,
   6.324555320336759: 96,
   6.4807406984078604: 192,
   6.6332495807107996: 96,
   6.7823299831252681: 192,
   6.9282032302755088: 32,
   7.0710678118654755: 336,
   7.2111025509279782: 96,
   7.3484692283495345: 384,
   7.4833147735478827: 192,
   7.6157731058639087: 96,
   7.8740078740118111: 384,
   8.0: 24,
   8.1240384046359608: 384,
   8.2462112512353212: 192,
   8.3666002653407556: 192,
   8.4852813742385695: 144,
   8.6023252670426267: 480,
   8.717797887081348: 96,
   8.8317608663278477: 192,
   8.9442719099991592: 96,
   9.0553851381374

In [10]:
# The minimum interatomic distance can be found once measurement has been done
# This can be useful for setting the potential length scale
([NaCl.contact(i, j) for i in range(NaCl.atom_types()) for j in range(NaCl.atom_types())],
 [CsCl.contact(i, j) for i in range(CsCl.atom_types()) for j in range(CsCl.atom_types())])

([1.4142135623730951, 1.0, 1.0, 1.4142135623730951],
 [2.0, 1.7320508075688772, 1.7320508075688772, 2.0])

Calculating Energies
--
Once measurements have been made, energies can be calculated.  The accuracy of the calculated energies will depend upon the range of the potential(s) used and the measurement cutoff.  Note that potentials and parameter sets are specified for all possible pairs of atom types.  Underspecification (missing pairs) or overspecification (pairs specified both as $(a,b)$ and $(b,a)$) will raise `ValueError`s.

One energy will be returned per parameter set.  Energies are calculated per atom, so using a periodic supercell in place of a unit cell should not affect the results provided all else is held constant.

In [11]:
NaCl_s = (2.0 ** (-1.0 / 6.0))
NaCl.energy({
    (0, 0): potential.lj_piecewise_repel, (1, 1): potential.lj_piecewise_repel, (0, 1): potential.lj_piecewise_full}, [{
    (0, 0): (NaCl_s, 1.0, 12.0, 0.0), (1, 1): (NaCl_s, 1.0, 12.0, 0.0), (0, 1): (NaCl_s, 1.0, 24.0, 12.0, 0.0)}, {
    (0, 0): (NaCl_s, 1.0, 12.0, 0.0), (1, 1): (NaCl_s, 1.0, 12.0, 0.0), (0, 1): (NaCl_s, 1.0, 12.0, 12.0, 0.0)}, {
    (0, 0): (NaCl_s, 1.0, 12.0, 0.0), (1, 1): (NaCl_s, 1.0, 12.0, 0.0), (0, 1): (NaCl_s, 1.0, 6.0, 12.0, 0.0)}])

[-3.0000061449370992, -3.0092336647264317, -3.5867436685091536]

In [12]:
CsCl_s = (3.0 ** 0.5) * (2.0 ** (-1.0 / 6.0))
CsCl.energy({
    (0, 0): potential.lj_piecewise_repel, (1, 1): potential.lj_piecewise_repel, (0, 1): potential.lj_piecewise_full}, [{
    (0, 0): (CsCl_s, 1.0, 12.0, 0.0), (1, 1): (CsCl_s, 1.0, 12.0, 0.0), (0, 1): (CsCl_s, 1.0, 24.0, 12.0, 0.0)}, {
    (0, 0): (CsCl_s, 1.0, 12.0, 0.0), (1, 1): (CsCl_s, 1.0, 12.0, 0.0), (0, 1): (CsCl_s, 1.0, 12.0, 12.0, 0.0)}, {
    (0, 0): (CsCl_s, 1.0, 12.0, 0.0), (1, 1): (CsCl_s, 1.0, 12.0, 0.0), (0, 1): (CsCl_s, 1.0, 6.0, 12.0, 0.0)}])

[-4.0000014729919879, -4.0073525380312667, -4.6905299740995288]

The behavior for the sharper potentials is quantitatively as expected given the properties of the NaCl and CsCl structures.  Note how the energy decreases as the potential is made softer (qualitatively as expected).

We can recalculate the energies out to a larger cutoff and observe the effects on their magnitudes.

In [13]:
NaCl.measure(20)
CsCl.measure(20)

In [14]:
NaCl.energy({
    (0, 0): potential.lj_piecewise_repel, (1, 1): potential.lj_piecewise_repel, (0, 1): potential.lj_piecewise_full}, [{
    (0, 0): (NaCl_s, 1.0, 12.0, 0.0), (1, 1): (NaCl_s, 1.0, 12.0, 0.0), (0, 1): (NaCl_s, 1.0, 24.0, 12.0, 0.0)}, {
    (0, 0): (NaCl_s, 1.0, 12.0, 0.0), (1, 1): (NaCl_s, 1.0, 12.0, 0.0), (0, 1): (NaCl_s, 1.0, 12.0, 12.0, 0.0)}, {
    (0, 0): (NaCl_s, 1.0, 12.0, 0.0), (1, 1): (NaCl_s, 1.0, 12.0, 0.0), (0, 1): (NaCl_s, 1.0, 6.0, 12.0, 0.0)}])

[-3.0000061449370992, -3.0092336651256315, -3.5886261333530283]

In [15]:
CsCl.energy({
    (0, 0): potential.lj_piecewise_repel, (1, 1): potential.lj_piecewise_repel, (0, 1): potential.lj_piecewise_full}, [{
    (0, 0): (CsCl_s, 1.0, 12.0, 0.0), (1, 1): (CsCl_s, 1.0, 12.0, 0.0), (0, 1): (CsCl_s, 1.0, 24.0, 12.0, 0.0)}, {
    (0, 0): (CsCl_s, 1.0, 12.0, 0.0), (1, 1): (CsCl_s, 1.0, 12.0, 0.0), (0, 1): (CsCl_s, 1.0, 12.0, 12.0, 0.0)}, {
    (0, 0): (CsCl_s, 1.0, 12.0, 0.0), (1, 1): (CsCl_s, 1.0, 12.0, 0.0), (0, 1): (CsCl_s, 1.0, 6.0, 12.0, 0.0)}])

[-4.0000014729919879, -4.0073525982407681, -4.7021870397030039]

Scaling
--
Note how the position of the potential minimum was explicitly set in the previous example.  We can alternatively scale a unit cell based on measurement information so that the potential need not be changed in this way.

In [16]:
# Determine the minimum interatomic distance
minimum = lambda cell: min(cell.contact(i, j) for i in range(cell.atom_types()) for j in range(cell.atom_types()))
(minimum(NaCl), minimum(CsCl))

(1.0, 1.7320508075688772)

In [17]:
# Perform the rescaling so that the distance is as desired
potential_minimum = 2 ** (1 / 6)
NaCl.rescale([NaCl.vector(i) * potential_minimum / minimum(NaCl) for i in range(NaCl.dimensions())])
CsCl.rescale([CsCl.vector(i) * potential_minimum / minimum(CsCl) for i in range(CsCl.dimensions())])

In [18]:
# This modifies the cell in place
([NaCl.vector(i) for i in range(NaCl.dimensions())], [CsCl.vector(i) for i in range(CsCl.dimensions())])

([array([ 2.2449241,  0.       ,  0.       ]),
  array([ 0.       ,  2.2449241,  0.       ]),
  array([ 0.       ,  0.       ,  2.2449241])],
 [array([ 1.29610753,  0.        ,  0.        ]),
  array([ 0.        ,  1.29610753,  0.        ]),
  array([ 0.        ,  0.        ,  1.29610753])])

In [19]:
# We must remeasure the cells after rescaling
NaCl.measure(10)
CsCl.measure(10)

In [20]:
# Note that the minima are now set as desired
(minimum(NaCl), minimum(CsCl))

(1.122462048309373, 1.122462048309373)

In [21]:
# The potentials can be applied without compensation
NaCl.energy({
    (0, 0): potential.lj_piecewise_repel, (1, 1): potential.lj_piecewise_repel, (0, 1): potential.lj_piecewise_full}, [{
    (0, 0): (1.0, 1.0, 12.0, 0.0), (1, 1): (1.0, 1.0, 12.0, 0.0), (0, 1): (1.0, 1.0, 24.0, 12.0, 0.0)}, {
    (0, 0): (1.0, 1.0, 12.0, 0.0), (1, 1): (1.0, 1.0, 12.0, 0.0), (0, 1): (1.0, 1.0, 12.0, 12.0, 0.0)}, {
    (0, 0): (1.0, 1.0, 12.0, 0.0), (1, 1): (1.0, 1.0, 12.0, 0.0), (0, 1): (1.0, 1.0, 6.0, 12.0, 0.0)}])

[-3.0000061449370992, -3.0092336640321569, -3.5858932244454409]

In [22]:
# You may notice that the energies deviate very slightly from the ones calculated before:
# this is due to the cutoff remaining constant but the cell size changing.  You should
# determine what cutoffs are most appropriate for your potentials and structures.
CsCl.energy({
    (0, 0): potential.lj_piecewise_repel, (1, 1): potential.lj_piecewise_repel, (0, 1): potential.lj_piecewise_full}, [{
    (0, 0): (1.0, 1.0, 12.0, 0.0), (1, 1): (1.0, 1.0, 12.0, 0.0), (0, 1): (1.0, 1.0, 24.0, 12.0, 0.0)}, {
    (0, 0): (1.0, 1.0, 12.0, 0.0), (1, 1): (1.0, 1.0, 12.0, 0.0), (0, 1): (1.0, 1.0, 12.0, 12.0, 0.0)}, {
    (0, 0): (1.0, 1.0, 12.0, 0.0), (1, 1): (1.0, 1.0, 12.0, 0.0), (0, 1): (1.0, 1.0, 6.0, 12.0, 0.0)}])

[-4.0000014729919862, -4.0073525969561965, -4.7000532092847767]

Low Level Interface Module Reference
--

In [23]:
help(crystal)

Help on module crystal:

NAME
    crystal - Contains low-level routines for crystal structure energy calculations.

CLASSES
    builtins.object
        Cell
    
    class Cell(builtins.object)
     |  Creates a unit cell with the specified parameters.
     |  
     |  Parameters
     |  ----------
     |  vectors : ndarray
     |      Cell vectors as a matrix of row vectors (columns set dimensions)
     |  atom_lists : list of ndarray
     |      Atom coordinates as matrices of row vectors (one matrix per type)
     |  
     |  Raises
     |  ------
     |  ValueError
     |      Inconsistent dimensions or invalid values of input data
     |  
     |  Methods defined here:
     |  
     |  __init__(self, vectors, atom_lists)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  atom(self, list_index, atom_index)
     |      Retrieves the coordinates of a given atom.
     |      
     |      Parameters
     |      ----------
     |      list_index 

In [24]:
help(potential)

Help on module potential:

NAME
    potential - Contains various pairwise potential models.

FUNCTIONS
    jagla(r, sigma, epsilon, n, s, a0, a1, a2, b0, b1, b2, q)
        Represents a generalized "Jagla" potential.
        
        Parameters
        ----------
        r : ndarray
            Separation distances
        sigma : float
            Power term length scale factor
        epsilon : float
            Power term energy scale factor
        n : float
            Power term exponent
        s : float
            Power term horizontal shift distance
        a0 : float
            First exponential term energy scale factor
        a1 : float
            First exponential term shape factor
        a2 : float
            First exponential term horizontal shift distance
        b0 : float
            Second exponential term energy scale factor
        b1 : float
            Second exponential term shape factor
        b2 : float
            Second exponential term horizontal shif